In [7]:
import sys
import torch
import logging
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from scipy.stats import pearsonr
from torch.utils.data import TensorDataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForSequenceClassification

logging.disable(logging.WARNING)
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)

In [8]:
# Load all the data
trainpath = 'data/train.csv'
testpath = 'data/test.csv'
valpath = 'data/validation.csv'

traindata = pd.read_csv(trainpath)
testdata = pd.read_csv(testpath)
valdata = pd.read_csv(valpath)

traindata['score'] = traindata['score'].apply(lambda x: (x)/5.0)
testdata['score'] = testdata['score'].apply(lambda x: (x)/5.0)
valdata['score'] = valdata['score'].apply(lambda x: (x)/5.0)

In [ ]:
def pearson_corr(y_true, y_pred):
    corr, _ = pearsonr(y_true, y_pred)
    return corr

In [10]:
# compute the sequence length using 95% samples logic
lengths = []
for _, row in traindata.iterrows():
    lengths.append(len(row['sentence1']))
    lengths.append(len(row['sentence2']))

lengths.sort()
MAX_LEN = lengths[int(0.95*len(lengths))]
print(MAX_LEN)

133


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def convert_sentences_to_features(sentences, tokenizer, max_len):
    input_ids = []
    attention_masks = []
    token_type_ids = []

    for i in range(0, len(sentences), 2):
        encoded_dict = tokenizer.encode_plus(sentences[i], sentences[i+1], add_special_tokens=True, max_length=max_len, truncation=True, padding='max_length', return_attention_mask=True, return_tensors='pt', truncation_strategy='longest_first')
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        token_type_ids.append(encoded_dict['token_type_ids'])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    token_type_ids = torch.cat(token_type_ids, dim=0)

    return input_ids, attention_masks, token_type_ids

### Fine-Tune BERT Model

In [12]:
# Hyperparameters
EPOCHS = 10
BATCH_SIZE = 32
LEARN_RATE = 1e-5

In [13]:
x_train = []
for _, row in traindata.iterrows():
    x_train.append(row['sentence1'])
    x_train.append(row['sentence2'])

input_ids, attention_masks, token_type_ids = convert_sentences_to_features(x_train, tokenizer, MAX_LEN)
y_train = torch.tensor(traindata['score'], dtype=torch.float)

trainset = TensorDataset(input_ids, attention_masks, token_type_ids, y_train)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

config = BertConfig.from_pretrained(
    'bert-base-uncased',
    num_labels=1,
    output_attentions=False,
    output_hidden_states=False,
)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)
optimizer = optim.Adam(model.parameters(), lr=LEARN_RATE, betas=[0.5, 0.99])
loss_fn = nn.MSELoss()

for epoch in range(EPOCHS):
    model.train()
    t_loss = 0
    for _, batch in enumerate(trainloader):
        input_ids, attention_masks, _, labels = tuple(t for t in batch)
        optimizer.zero_grad()
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)
        loss = outputs[0]
        loss.backward()
        optimizer.step()
        t_loss += loss.item()
    print(f'Epoch: {epoch}\tLoss: {t_loss / len(trainloader)}')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Epoch: 0	Loss: 0.057592995889070964
Epoch: 1	Loss: 0.039322442406167585
Epoch: 2	Loss: 0.0325561978129877
Epoch: 3	Loss: 0.02672251861852904
Epoch: 4	Loss: 0.022323650737396544
Epoch: 5	Loss: 0.019402472633454536
Epoch: 6	Loss: 0.01727930448897597
Epoch: 7	Loss: 0.014879244353829158
Epoch: 8	Loss: 0.013136854858344628
Epoch: 9	Loss: 0.011161395475371844


In [24]:
model_untrained = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

x_val = []
for _, row in valdata.iterrows():
    x_val.append(row['sentence1'])
    x_val.append(row['sentence2'])

input_ids, attention_masks, token_type_ids = convert_sentences_to_features(x_val, tokenizer, MAX_LEN)
y_val = torch.tensor(valdata['score'], dtype=torch.float)
valset = TensorDataset(input_ids, attention_masks, token_type_ids, y_val)
valloader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False)

y_true, y_pred = [], []
with torch.no_grad():
    for _, batch in enumerate(valloader):
        input_ids, attention_masks, _, labels = tuple(t for t in batch)
        outputs = model_untrained(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)
        y_true.extend(labels.tolist())
        y_pred.extend([row[0] for row in outputs[1].tolist()])

corr = pearson_corr(y_true, y_pred)
print("Pearson correlation coefficient: {:.2f}".format(corr))

Pearson correlation coefficient: -0.14


In [20]:
x_val = []
for _, row in valdata.iterrows():
    x_val.append(row['sentence1'])
    x_val.append(row['sentence2'])

input_ids, attention_masks, token_type_ids = convert_sentences_to_features(x_val, tokenizer, MAX_LEN)
y_val = torch.tensor(valdata['score'], dtype=torch.float)
valset = TensorDataset(input_ids, attention_masks, token_type_ids, y_val)
valloader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False)

y_true, y_pred = [], []
with torch.no_grad():
    for _, batch in enumerate(valloader):
        input_ids, attention_masks, _, labels = tuple(t for t in batch)
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)
        y_true.extend(labels.tolist())
        y_pred.extend([row[0] for row in outputs[1].tolist()])

corr = pearson_corr(y_true, y_pred)
print("Pearson correlation coefficient: {:.2f}".format(corr))

Pearson correlation coefficient: 0.83


In [25]:
model_untrained = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

x_test = []
for _, row in testdata.iterrows():
    x_test.append(row['sentence1'])
    x_test.append(row['sentence2'])

input_ids, attention_masks, token_type_ids = convert_sentences_to_features(x_test, tokenizer, MAX_LEN)
y_test = torch.tensor(testdata['score'], dtype=torch.float)

testset = TensorDataset(input_ids, attention_masks, token_type_ids, y_test)
testloader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False)

y_true, y_pred = [], []
with torch.no_grad():
    for _, batch in enumerate(testloader):
        input_ids, attention_masks, _, labels = tuple(t for t in batch)
        outputs = model_untrained(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)
        y_true.extend(labels.tolist())
        y_pred.extend([row[0] for row in outputs[1].tolist()])

corr = pearson_corr(y_true, y_pred)
print("Pearson correlation coefficient: {:.2f}".format(corr))

Pearson correlation coefficient: 0.16


In [23]:
x_test = []
for _, row in testdata.iterrows():
    x_test.append(row['sentence1'])
    x_test.append(row['sentence2'])

input_ids, attention_masks, token_type_ids = convert_sentences_to_features(x_test, tokenizer, MAX_LEN)
y_test = torch.tensor(testdata['score'], dtype=torch.float)

testset = TensorDataset(input_ids, attention_masks, token_type_ids, y_test)
testloader = DataLoader(valset, batch_size=BATCH_SIZE, shuffle=False)

y_true, y_pred = [], []
with torch.no_grad():
    for _, batch in enumerate(testloader):
        input_ids, attention_masks, _, labels = tuple(t for t in batch)
        outputs = model(input_ids, token_type_ids=None, attention_mask=attention_masks, labels=labels)
        y_true.extend(labels.tolist())
        y_pred.extend([row[0] for row in outputs[1].tolist()])

corr = pearson_corr(y_true, y_pred)
print("Pearson correlation coefficient: {:.2f}".format(corr))

Pearson correlation coefficient: 0.82
